In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import quantities as pq
from bsb import Scaffold, parse_configuration_file
from scipy.signal import welch, find_peaks
import pandas as pd
from IPython.display import HTML, display
from utils import extract_device_data, plot_signal

In [ ]:
# bsb.options.verbosity = 3
config = parse_configuration_file("configurations/io.yaml", parser="yaml")

scaffold = Scaffold(config)
scaffold.compile(clear=True)

In [ ]:
storage = scaffold.storage

for cell_name, cell_type in scaffold.cell_types.items():
    ps = cell_type.get_placement_set() 
    pos = ps.load_positions()
    print(f"{len(pos)} {cell_name} placed")

In [ ]:
cs = scaffold.get_connectivity_set("gap_io")
for pre_locs, post_locs in cs.load_connections():
    pre_cell_id = pre_locs[0]
    post_cell_id = post_locs[0]

    print(f"Gap junction between cell {pre_cell_id} and {post_cell_id}")

In [ ]:
results = scaffold.run_simulation("basal_activity")
spiketrains = results.spiketrains
signals = results.analogsignals

In [ ]:
# Extract device data
devices_data = extract_device_data(config)

# Create a DataFrame for better display
devices_df = pd.DataFrame(devices_data)
HTML(devices_df.to_html(index=False))


In [ ]:
# Analyze signal without input
plot_signal(signal=signals[0], title="IO EGLIF Membrane Potential")

In [ ]:
# Analyze signal with input
plot_signal(signal=signals[1], devices=devices_data)

In [ ]:
# Analyze input region
plot_signal(signal=signals[1], devices=devices_data, start=400, stop=700, figsize=(5, 4))

In [ ]:
# -------------------------
# NOT WORKING: KeyError: 'senders'
# -------------------------

nb_spike_trains = len(spiketrains)
fig, ax = plt.subplots(nb_spike_trains, sharex=True, figsize=(10, nb_spike_trains * 6))

if nb_spike_trains == 1:
    ax = np.array([ax])  # Wrap in array to make subscriptable

for i, spike_t in enumerate(spiketrains):  # Iterate over all spike trains
    name = spike_t.annotations["device"]  # Retrieve the device name
    cell_list = spike_t.annotations["senders"]  # Retrieve the ids of the cells spiking
    spike_times = spike_t.magnitude  # Retrieve the spike times
    ax[i].scatter(spike_times, cell_list, c=f"C{i}")
    ax[i].set_xlabel(f"Time ({spike_t.times.units.dimensionality.string})")
    ax[i].set_ylabel("Neuron ID")
    ax[i].set_title(f"Spikes from {name}")

In [ ]:
# Store results in lists
results = []

for i, vm_signal in enumerate(signals):
    sampling_rate = vm_signal.sampling_rate.rescale('Hz').magnitude
    v_m = vm_signal.magnitude.flatten()

    # Compute power spectral density
    frequencies, power = welch(v_m, fs=sampling_rate, nperseg=2048)

    # Focus on STO frequency range (1–100 Hz)
    sto_band = (frequencies >= 1) & (frequencies <= 100)
    sto_freq = frequencies[sto_band][np.argmax(power[sto_band])]

    # Find peaks and troughs
    peaks, _ = find_peaks(v_m)
    troughs, _ = find_peaks(-v_m)

    # Make sure we align peaks/troughs
    min_len = min(len(peaks), len(troughs))
    amplitudes = np.abs(v_m[peaks[:min_len]] - v_m[troughs[:min_len]])
    mean_amp = np.mean(amplitudes)
    
    # Store results
    results.append({
        "Signal ID": i,
        "STO Frequency (Hz)": round(sto_freq, 2),
        "Mean STO Amplitude (mV)": round(mean_amp, 2),
        "Peak Count": len(peaks),
        "Trough Count": len(troughs)
    })

# Create DataFrame
results_df = pd.DataFrame(results)

# Display as formatted table
print("IO Cell Analysis")
print("=" * 50)
HTML(results_df.to_html(index=False))